# Chapter 18: Mapped Types

---

## 18.1 Understanding Mapped Types

Mapped types allow you to create new types by transforming properties of an existing type. They work similarly to array mapping in JavaScript, but operate at the type level, creating new object types by iterating over keys.

### 18.1.1 Basic Mapped Type Syntax

**The Anatomy of a Mapped Type:**

```typescript
// Basic mapped type syntax
type MappedType<T> = {
  [K in keyof T]: TransformedType
};

// Simple example: Make all properties optional
type MyPartial<T> = {
  [K in keyof T]?: T[K]
};

// Usage
interface User {
  name: string;
  age: number;
  email: string;
}

type PartialUser = MyPartial<User>;
// Result: { name?: string; age?: number; email?: string }
```

**Mapped Type Syntax Breakdown:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Mapped Type Syntax                                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   type NewType<T> = {                                               │
│     [K in keyof T]: T[K];      ← Property signature                 │
│     │ │    │        │                                               │
│     │ │    │        └── Type of the property (from original)       │
│     │ │    └────────── Union of keys from T (keyof T)              │
│     │ └─────────────── Iteration over keys (like for...in)         │
│     └───────────────── Property key variable                        │
│   };                                                                │
│                                                                     │
│   Components:                                                       │
│   • [K in keyof T] - Iterates over all keys in T                    │
│   • K - Represents each key (string | number | symbol)              │
│   • keyof T - Union of all keys in T                                │
│   • T[K] - Lookup type (value type for key K)                       │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 18.1.2 How Mapping Works

**Step-by-Step Transformation:**

```typescript
// Original type
interface Person {
  name: string;
  age: number;
  isActive: boolean;
}

// Mapped type that keeps values the same
type Clone<T> = {
  [K in keyof T]: T[K]
};

// TypeScript processes this as:
// 1. keyof Person = "name" | "age" | "isActive"
// 2. For each key K in that union:
//    - K="name":  { name: Person["name"] }  → { name: string }
//    - K="age":   { age: Person["age"] }    → { age: number }
//    - K="isActive": { isActive: Person["isActive"] } → { isActive: boolean }
// 3. Merge all into: { name: string; age: number; isActive: boolean }

type PersonClone = Clone<Person>; // Same as Person

// Transforming values
type Stringify<T> = {
  [K in keyof T]: string
};

type StringifiedPerson = Stringify<Person>;
// { name: string; age: string; isActive: string }
// All values become string regardless of original type

// Transforming keys (key remapping)
type Getters<T> = {
  [K in keyof T as `get${Capitalize<string & K>}`]: () => T[K]
};

type PersonGetters = Getters<Person>;
// {
//   getName: () => string;
//   getAge: () => number;
//   getIsActive: () => boolean;
// }
```

---

## 18.2 Built-in Mapped Types

TypeScript provides several built-in mapped types that are essential for everyday development.

### 18.2.1 `Partial<T>` - Make All Properties Optional

**Implementation and Usage:**

```typescript
// Built-in implementation
type Partial<T> = {
  [P in keyof T]?: T[P];
};

// Usage
interface Todo {
  title: string;
  description: string;
  completed: boolean;
  dueDate: Date;
}

// All properties become optional
type PartialTodo = Partial<Todo>;

const updateTodo = (todo: Todo, fields: PartialTodo): Todo => {
  return { ...todo, ...fields };
};

const todo: Todo = {
  title: "Learn TypeScript",
  description: "Study mapped types",
  completed: false,
  dueDate: new Date("2024-12-31")
};

// Can update just one field
const updated = updateTodo(todo, { completed: true });
// Or multiple fields
const updated2 = updateTodo(todo, { 
  description: "Study advanced TypeScript",
  dueDate: new Date("2024-11-30")
});

// Practical use: Function options with defaults
interface Config {
  host: string;
  port: number;
  ssl: boolean;
  timeout: number;
}

function createConnection(config: Partial<Config> = {}): Config {
  return {
    host: config.host ?? "localhost",
    port: config.port ?? 5432,
    ssl: config.ssl ?? false,
    timeout: config.timeout ?? 5000
  };
}

// Can provide partial config
const conn = createConnection({ host: "prod.db.com", ssl: true });
```

### 18.2.2 `Required<T>` - Make All Properties Required

**Implementation and Usage:**

```typescript
// Built-in implementation
type Required<T> = {
  [P in keyof T]-?: T[P];  // -? removes optionality
};

// Usage
interface UserProfile {
  name: string;
  bio?: string;
  avatar?: string;
  website?: string;
}

// All properties become required
type CompleteProfile = Required<UserProfile>;

// Must provide all properties
const profile: CompleteProfile = {
  name: "John",
  bio: "Developer",      // Required now
  avatar: "avatar.jpg",  // Required now
  website: "example.com" // Required now
};

// Practical use: Validating complete objects
function validateProfile(profile: Required<UserProfile>): boolean {
  return profile.bio.length > 0 && 
         profile.avatar.length > 0 &&
         profile.website.startsWith("http");
}

// Combining with Partial
type EnsureComplete<T> = Required<Partial<T>>; // All optional but required if present
```

### 18.2.3 `Readonly<T>` - Make All Properties Readonly

**Implementation and Usage:**

```typescript
// Built-in implementation
type Readonly<T> = {
  readonly [P in keyof T]: T[P];
};

// Usage
interface Point {
  x: number;
  y: number;
}

type ImmutablePoint = Readonly<Point>;

const point: ImmutablePoint = { x: 10, y: 20 };
// point.x = 30; // ❌ Error: Cannot assign to 'x' because it is a read-only property

// Practical use: Immutable state
interface AppState {
  user: { name: string };
  theme: "light" | "dark";
  notifications: string[];
}

function createImmutableState(state: AppState): Readonly<AppState> {
  return Object.freeze({ ...state });
}

const state = createImmutableState({
  user: { name: "John" },
  theme: "dark",
  notifications: []
});

// state.theme = "light"; // ❌ Error: readonly property

// Deep readonly (custom implementation)
type DeepReadonly<T> = {
  readonly [P in keyof T]: T[P] extends object 
    ? DeepReadonly<T[P]> 
    : T[P];
};

const deepState: DeepReadonly<{ nested: { value: number } }> = {
  nested: { value: 42 }
};
// deepState.nested.value = 10; // ❌ Error: nested is readonly, value is readonly
```

### 18.2.4 `Pick<T, K>` - Select Specific Properties

**Implementation and Usage:**

```typescript
// Built-in implementation
type Pick<T, K extends keyof T> = {
  [P in K]: T[P];
};

// Usage
interface User {
  id: number;
  name: string;
  email: string;
  password: string;
  createdAt: Date;
}

// Pick only public-safe fields
type PublicUser = Pick<User, "id" | "name" | "email">;
// { id: number; name: string; email: string }

const publicUser: PublicUser = {
  id: 1,
  name: "John",
  email: "john@example.com"
  // password and createdAt are excluded
};

// Practical use: API response shaping
function getUserProfile(user: User): Pick<User, "name" | "email"> {
  return {
    name: user.name,
    email: user.email
  };
}

// Dynamic picking with union
type UserFields = "name" | "email" | "id";
type SelectedUser = Pick<User, UserFields>;
```

### 18.2.5 `Omit<T, K>` - Remove Specific Properties

**Implementation and Usage:**

```typescript
// Built-in implementation (uses Pick and Exclude)
type Omit<T, K extends keyof any> = Pick<T, Exclude<keyof T, K>>;

// Usage
interface User {
  id: number;
  name: string;
  email: string;
  password: string;
  internalNotes: string;
}

// Remove sensitive fields
type SafeUser = Omit<User, "password" | "internalNotes">;
// { id: number; name: string; email: string }

const safeUser: SafeUser = {
  id: 1,
  name: "John",
  email: "john@example.com"
};

// Practical use: Creating DTOs
type CreateUserDTO = Omit<User, "id" | "createdAt">;
// For creating new users, client shouldn't send id

// Omit vs Pick - when to use each
// Use Pick when you want FEW properties from MANY
// Use Omit when you want MANY properties except FEW

// Combining Pick and Omit
type UpdateUserDTO = Partial<Omit<User, "id" | "createdAt">>;
// All fields optional except id and createdAt
```

### 18.2.6 `Record<K, T>` - Create Object Type with Specific Keys

**Implementation and Usage:**

```typescript
// Built-in implementation
type Record<K extends keyof any, T> = {
  [P in K]: T;
};

// Usage
// Create object with specific keys and uniform values
type PageNames = "home" | "about" | "contact";
type PageInfo = { title: string; path: string };

const pages: Record<PageNames, PageInfo> = {
  home: { title: "Home", path: "/" },
  about: { title: "About", path: "/about" },
  contact: { title: "Contact", path: "/contact" }
  // All keys required, all values must be PageInfo
};

// Practical use: Dictionaries and Maps
type UserId = string;
type UserCache = Record<UserId, User>;

const cache: UserCache = {
  "user-1": { id: 1, name: "John", email: "john@example.com" },
  "user-2": { id: 2, name: "Jane", email: "jane@example.com" }
};

// Type-safe access
const user = cache["user-1"]; // Type: User (not User | undefined)

// Dynamic record with string keys
type StringDictionary<T> = Record<string, T>;

const config: StringDictionary<number> = {
  timeout: 5000,
  retries: 3,
  port: 8080
};
```

---

## 18.3 Creating Custom Mapped Types

Beyond the built-in types, you can create powerful custom mapped types for specific use cases.

### 18.3.1 Nullable Types

**Making All Properties Nullable:**

```typescript
// Make all properties nullable
type Nullable<T> = {
  [P in keyof T]: T[P] | null;
};

interface User {
  name: string;
  age: number;
}

type NullableUser = Nullable<User>;
// { name: string | null; age: number | null }

// Make all properties nullable and optional
type OptionalNullable<T> = {
  [P in keyof T]?: T[P] | null;
};

// Nullify specific types only
type NullifyStrings<T> = {
  [P in keyof T]: T[P] extends string ? T[P] | null : T[P];
};

interface Mixed {
  name: string;
  count: number;
  active: boolean;
}

type Nullified = NullifyStrings<Mixed>;
// { name: string | null; count: number; active: boolean }
```

### 18.3.2 Event Handler Types

**Creating Event Handler Interfaces:**

```typescript
// Create event handler interface from data interface
type EventHandlers<T> = {
  [K in keyof T as `on${Capitalize<string & K>}`]: (value: T[K]) => void;
};

interface UserEvents {
  login: { userId: string; timestamp: Date };
  logout: { userId: string };
  update: { field: string; value: unknown };
}

type UserEventHandlers = EventHandlers<UserEvents>;
// {
//   onLogin: (value: { userId: string; timestamp: Date }) => void;
//   onLogout: (value: { userId: string }) => void;
//   onUpdate: (value: { field: string; value: unknown }) => void;
// }

// Implementation
class UserEventEmitter implements UserEventHandlers {
  onLogin(value: { userId: string; timestamp: Date }): void {
    console.log(`User ${value.userId} logged in`);
  }
  
  onLogout(value: { userId: string }): void {
    console.log(`User ${value.userId} logged out`);
  }
  
  onUpdate(value: { field: string; value: unknown }): void {
    console.log(`Updated ${value.field}`);
  }
}
```

### 18.3.3 API Response Types

**Transforming for API Responses:**

```typescript
// Wrap all properties in ApiResponse
type ApiResponse<T> = {
  [K in keyof T]: {
    data: T[K];
    status: "success" | "error";
    timestamp: Date;
  };
};

interface Services {
  getUser: User;
  getProducts: Product[];
  getOrder: Order;
}

type ApiServices = ApiResponse<Services>;
// {
//   getUser: { data: User; status: "success" | "error"; timestamp: Date; };
//   getProducts: { data: Product[]; status: "success" | "error"; timestamp: Date; };
//   getOrder: { data: Order; status: "success" | "error"; timestamp: Date; };
// }

// Flatten nested objects for forms
type FormFields<T> = {
  [K in keyof T]: T[K] extends object 
    ? { [P in keyof T[K] as `${string & K}.${string & P}`]: T[K][P] }
    : { [P in K]: T[P] }
}[keyof T];

interface NestedConfig {
  database: { host: string; port: number };
  cache: { enabled: boolean; ttl: number };
}

type FlatConfig = FormFields<NestedConfig>;
// { "database.host": string; "database.port": number; "cache.enabled": boolean; "cache.ttl": number }
```

---

## 18.4 Key Remapping via `as`

TypeScript 4.1+ allows remapping keys using the `as` clause in mapped types, enabling powerful key transformations.

### 18.4.1 Renaming Properties

**Key Transformation Patterns:**

```typescript
// Add prefix to all keys
type AddPrefix<T, P extends string> = {
  [K in keyof T as `${P}${string & K}`]: T[K];
};

interface User {
  name: string;
  age: number;
}

type PrefixedUser = AddPrefix<User, "user_">;
// { user_name: string; user_age: number }

// Add suffix
type AddSuffix<T, S extends string> = {
  [K in keyof T as `${string & K}${S}`]: T[K];
};

type SuffixedUser = AddSuffix<User, "_field">;
// { name_field: string; age_field: number }

// Change case (using template literal types)
type SnakeCase<T> = {
  [K in keyof T as SnakeCaseKey<string & K>]: T[K];
};

type SnakeCaseKey<S extends string> = S extends `${infer T}${infer U}`
  ? `${Lowercase<T>}${SnakeCaseRest<U>}`
  : S;

type SnakeCaseRest<S extends string> = S extends `${infer T}${infer U}`
  ? T extends Uppercase<T>
    ? `_${Lowercase<T>}${SnakeCaseRest<U>}`
    : `${T}${SnakeCaseRest<U>}`
  : S;

interface CamelCase {
  firstName: string;
  lastName: string;
  userAge: number;
}

type SnakeCaseProps = SnakeCase<CamelCase>;
// { first_name: string; last_name: string; user_age: number }
```

### 18.4.2 Filtering Properties

**Selective Key Mapping:**

```typescript
// Filter by value type
type PickByType<T, U> = {
  [K in keyof T as T[K] extends U ? K : never]: T[K];
};

interface User {
  name: string;
  age: number;
  email: string;
  isActive: boolean;
  createdAt: Date;
}

type StringProps = PickByType<User, string>;
// { name: string; email: string }

type NumberProps = PickByType<User, number>;
// { age: number }

// Filter out specific types
type OmitByType<T, U> = {
  [K in keyof T as T[K] extends U ? never : K]: T[K];
};

type NonStringProps = OmitByType<User, string>;
// { age: number; isActive: boolean; createdAt: Date }

// Filter by key pattern
type PickByPrefix<T, P extends string> = {
  [K in keyof T as K extends `${P}${string}` ? K : never]: T[K];
};

interface Prefixed {
  user_name: string;
  user_age: number;
  admin_role: string;
  admin_level: number;
}

type UserProps = PickByPrefix<Prefixed, "user_">;
// { user_name: string; user_age: number }
```

### 18.4.3 Template Literal Key Mapping

**Advanced Key Transformations:**

```typescript
// Create getter/setter pairs
type Accessors<T> = {
  [K in keyof T as `get${Capitalize<string & K>}`]: () => T[K];
} & {
  [K in keyof T as `set${Capitalize<string & K>}`]: (value: T[K]) => void;
};

interface Person {
  name: string;
  age: number;
}

type PersonAccessors = Accessors<Person>;
// {
//   getName: () => string;
//   getAge: () => number;
//   setName: (value: string) => void;
//   setAge: (value: number) => void;
// }

// Create CSS variable types from theme
type CSSVariables<T, P extends string = "--"> = {
  [K in keyof T as `${P}${KebabCase<string & K>}`]: string;
};

type KebabCase<S extends string> = S extends `${infer T}${infer U}`
  ? `${Lowercase<T>}${KebabCaseRest<U>}`
  : S;

type KebabCaseRest<S extends string> = S extends `${infer T}${infer U}`
  ? T extends Uppercase<T>
    ? `-${Lowercase<T>}${KebabCaseRest<U>}`
    : `${T}${KebabCaseRest<U>}`
  : S;

interface Theme {
  primaryColor: string;
  backgroundColor: string;
  fontSize: number;
}

type ThemeCSS = CSSVariables<Theme>;
// {
//   "--primary-color": string;
//   "--background-color": string;
//   "--font-size": string;
// }
```

---

## 18.5 Mapping Modifiers

Mapped types can add or remove modifiers like `readonly` and `?` (optional) using `+` and `-` prefixes.

### 18.5.1 Adding and Removing `readonly`

**Readonly Modifiers:**

```typescript
interface Mutable {
  name: string;
  count: number;
}

// Add readonly (+readonly or just readonly)
type Immutable1 = {
  +readonly [P in keyof Mutable]: Mutable[P];
};

type Immutable2 = {
  readonly [P in keyof Mutable]: Mutable[P];
};
// Both produce: { readonly name: string; readonly count: number }

// Remove readonly (-readonly)
interface ImmutableOriginal {
  readonly name: string;
  readonly count: number;
}

type MutableAgain = {
  -readonly [P in keyof ImmutableOriginal]: ImmutableOriginal[P];
};
// { name: string; count: number } - no longer readonly

// Toggle readonly (remove if present, add if not)
type ToggleReadonly<T> = {
  [P in keyof T]: T[P];
} & {
  -readonly [P in keyof T]: T[P];
};

// Practical: Create writable copy utility
function createMutable<T>(obj: T): { -readonly [P in keyof T]: T[P] } {
  return { ...obj };
}

const frozen: Readonly<{ x: number; y: number }> = { x: 1, y: 2 };
const mutable = createMutable(frozen);
mutable.x = 10; // OK - no longer readonly
```

### 18.5.2 Adding and Removing Optionality

**Optional Modifiers:**

```typescript
interface Required {
  name: string;
  age: number;
}

// Add optional (+? or just ?)
type Optional1 = {
  [P in keyof Required]+?: Required[P];
};

type Optional2 = {
  [P in keyof Required]?: Required[P];
};
// Both produce: { name?: string; age?: number }

// Remove optional (-?)
interface OptionalOriginal {
  name?: string;
  age?: number;
}

type RequiredAgain = {
  [P in keyof OptionalOriginal]-?: OptionalOriginal[P];
};
// { name: string; age: number } - no longer optional

// Practical: Required at certain stages
type Draft<T> = {
  [P in keyof T]+?: T[P]; // All optional
};

type Final<T> = {
  [P in keyof T]-?: T[P]; // All required
};

interface Article {
  title: string;
  content: string;
  author: string;
}

const draft: Draft<Article> = {
  title: "Draft Title"
  // content and author optional
};

const final: Final<Article> = {
  title: "Final Title",
  content: "Full content",
  author: "John Doe"
  // All required
};
```

### 18.5.3 Combining Modifiers

**Complex Modifier Patterns:**

```typescript
// Make all properties readonly and optional
type ReadonlyPartial<T> = {
  +readonly [P in keyof T]+?: T[P];
};

// Remove readonly and make required
type WritableRequired<T> = {
  -readonly [P in keyof T]-?: T[P];
};

// Deep modifiers
type DeepPartial<T> = {
  [P in keyof T]+?: T[P] extends object ? DeepPartial<T[P]> : T[P];
};

type DeepRequired<T> = {
  [P in keyof T]-?: T[P] extends object ? DeepRequired<T[P]> : T[P];
};

type DeepReadonly<T> = {
  +readonly [P in keyof T]: T[P] extends object ? DeepReadonly<T[P]> : T[P];
};

// Usage with nested objects
interface Nested {
  user: {
    profile: {
      name: string;
      settings: {
        theme: string;
      };
    };
  };
}

type DeepPartialNested = DeepPartial<Nested>;
// All properties at all levels become optional

// Conditional modifiers based on value type
type MakeNullableStrings<T> = {
  [P in keyof T]: T[P] extends string ? T[P] | null : T[P];
};
```

---

## 18.6 Homomorphic Mapped Types

A mapped type is homomorphic if it simply copies the property modifiers (readonly and optional) from the original type. Most built-in mapped types are homomorphic.

### 18.6.1 Understanding Homomorphism

**Preserving Structure:**

```typescript
// Homomorphic mapped type (preserves modifiers)
type Homomorphic<T> = {
  [P in keyof T]: T[P]
};

interface Original {
  readonly name: string;
  age?: number;
  readonly email?: string;
}

type Mapped = Homomorphic<Original>;
// Preserves:
// - name remains readonly
// - age remains optional
// - email remains readonly AND optional

// Non-homomorphic mapped type (loses modifiers)
type NonHomomorphic<T> = {
  [P in keyof T as string & P]: T[P]
};

type MappedNonHomo = NonHomomorphic<Original>;
// Loses readonly and optional:
// { name: string; age: number; email: string }

// Another non-homomorphic example
type WithPrefix<T> = {
  [P in keyof T as `prefix_${string & P}`]: T[P];
};

// Key remapping breaks homomorphism
```

### 18.6.2 Preserving Modifiers

**Maintaining Type Integrity:**

```typescript
// Homomorphic types preserve:
// 1. Optionality (?)
// 2. Readonly (readonly)
// 3. Property descriptors

// To preserve modifiers while transforming:
type TransformValues<T, U> = {
  [P in keyof T]: T[P] extends string ? U : T[P];
};

interface Mixed {
  readonly name: string;
  count?: number;
  readonly id: number;
}

type StringToNumber = TransformValues<Mixed, number>;
// {
//   readonly name: number;  // Still readonly
//   count?: number;          // Still optional
//   readonly id: number;     // Still readonly (but wasn't string so unchanged)
// }

// Breaking homomorphism intentionally
type ForceRequired<T> = {
  [P in keyof T]-?: T[P]  // -? removes optionality
};

type ForceMutable<T> = {
  -readonly [P in keyof T]: T[P]  // -readonly removes readonly
};
```

---

## 18.7 Practical Applications

### 18.7.1 Form Handling

**Type-Safe Forms:**

```typescript
// Create form state from data model
type FormState<T> = {
  [P in keyof T]: {
    value: T[P];
    touched: boolean;
    error?: string;
  };
};

interface User {
  name: string;
  email: string;
  age: number;
}

type UserForm = FormState<User>;
// {
//   name: { value: string; touched: boolean; error?: string; };
//   email: { value: string; touched: boolean; error?: string; };
//   age: { value: number; touched: boolean; error?: string; };
// }

// Create validation rules
type ValidationRules<T> = {
  [P in keyof T]?: (value: T[P]) => string | undefined;
};

const userValidation: ValidationRules<User> = {
  name: (value) => value.length < 3 ? "Name too short" : undefined,
  email: (value) => !value.includes("@") ? "Invalid email" : undefined,
  age: (value) => value < 18 ? "Must be 18+" : undefined
};

// Form initialization
function initializeForm<T>(initialValues: T): FormState<T> {
  const form = {} as FormState<T>;
  
  for (const key in initialValues) {
    form[key] = {
      value: initialValues[key],
      touched: false
    };
  }
  
  return form;
}

const form = initializeForm<User>({
  name: "",
  email: "",
  age: 0
});
```

### 18.7.2 API Type Safety

**End-to-End Type Safety:**

```typescript
// Define API endpoints
type Endpoints = {
  "/users": {
    GET: { response: User[] };
    POST: { body: Omit<User, "id">; response: User };
  };
  "/users/:id": {
    GET: { response: User };
    PUT: { body: Partial<User>; response: User };
    DELETE: { response: void };
  };
};

// Extract route parameters
type RouteParams<T extends string> = 
  T extends `${infer _Start}:${infer Param}/${infer Rest}`
    ? { [K in Param]: string } & RouteParams<Rest>
    : T extends `${infer _Start}:${infer Param}`
      ? { [K in Param]: string }
      : {};

type UserParams = RouteParams<"/users/:id">; // { id: string }

// Type-safe fetch wrapper
async function apiFetch<
  Path extends keyof Endpoints,
  Method extends keyof Endpoints[Path]
>(
  path: Path,
  method: Method,
  ...args: Method extends "GET" | "DELETE" 
    ? [] 
    : [body: Endpoints[Path][Method] extends { body: infer B } ? B : never]
): Promise<
  Endpoints[Path][Method] extends { response: infer R } ? R : never
> {
  // Implementation...
  return fetch(path, { method, body: JSON.stringify(args[0]) }) as any;
}

// Usage with full type safety
const users = await apiFetch("/users", "GET"); // User[]
const newUser = await apiFetch("/users", "POST", { name: "John", email: "john@example.com" }); // User
```

---

## 18.8 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored mapped types comprehensively:

**Key Takeaways:**

1. **Basic Syntax**: `[K in keyof T]` iterates over keys, `T[K]` looks up values
2. **Built-in Types**: `Partial`, `Required`, `Readonly`, `Pick`, `Omit`, `Record`
3. **Key Remapping**: Use `as` clause to transform keys (TypeScript 4.1+)
4. **Modifiers**: `+`/` -` to add/remove `readonly` and `?`
5. **Homomorphism**: Standard mapped types preserve modifiers; key remapping breaks it
6. **Practical Uses**: Forms, APIs, event handlers, CSS variables

### Practical Exercises

**Exercise 1: Built-in Mapped Types**

Create utility types:

```typescript
// 1. Implement your own version of Partial<T>

// 2. Implement your own version of Required<T>

// 3. Implement your own version of Readonly<T>

// 4. Implement your own version of Pick<T, K>

// 5. Implement your own version of Omit<T, K>

// 6. Implement your own version of Record<K, T>

// 7. Test each with example interfaces
```

**Exercise 2: Custom Mapped Types**

Build domain-specific utilities:

```typescript
// 1. Create Nullable<T> that makes all properties nullable

// 2. Create Mutable<T> that removes readonly from all properties

// 3. Create Stringify<T> that converts all property types to string

// 4. Create Promisify<T> that wraps all property types in Promise

// 5. Create EventHandlers<T> that creates onEvent handlers for each property

// 6. Create DeepPartial<T> that works recursively
```

**Exercise 3: Key Remapping**

Transform keys creatively:

```typescript
// 1. Create CamelCase<S> that converts snake_case to camelCase
//    type CamelCase<"user_name"> = "userName"

// 2. Create AddPrefix<T, P> that adds prefix to all keys
//    type Prefixed = AddPrefix<{name: string}, "user_">
//    // { user_name: string }

// 3. Create FilterByType<T, U> that keeps only keys of type U
//    type StringsOnly = FilterByType<{name: string, age: number}, string>
//    // { name: string }

// 4. Create CSSVariables<T> that converts property names to CSS custom properties
//    type ThemeVars = CSSVariables<{primaryColor: string}>
//    // { "--primary-color": string }
```

**Exercise 4: Real-World Application**

Build a type-safe state management system:

```typescript
// 1. Define a State interface with nested properties

// 2. Create Action<T> type that generates action creators for each property
//    { setName: (value: string) => Action; setAge: (value: number) => Action }

// 3. Create Selector<T> type that generates selectors for each property
//    { selectName: (state: State) => string; selectAge: (state: State) => number }

// 4. Create a function createStore<T>(initialState: T) that returns:
//    - state: DeepReadonly<T>
//    - actions: Action<T>
//    - selectors: Selector<T>

// 5. Test with a complex nested state object
```

**Exercise 5: Advanced Patterns**

Implement sophisticated type transformations:

```typescript
// 1. Create Flatten<T> that flattens nested objects into dot-notation keys
//    Flatten<{user: {name: string}}> = {"user.name": string}

// 2. Create Unflatten<T> that reverses the above operation

// 3. Create Diff<T, U> that shows only differing properties between two types

// 4. Create Merge<T, U> that intelligently merges two object types

// 5. Create Validate<T, Schema> that checks if T satisfies validation rules
```

### Additional Resources

- **TypeScript Handbook - Mapped Types**: https://www.typescriptlang.org/docs/handbook/2/mapped-types.html
- **Key Remapping**: https://www.typescriptlang.org/docs/handbook/release-notes/typescript-4-1.html#key-remapping-in-mapped-types
- **Template Literal Types**: https://www.typescriptlang.org/docs/handbook/2/template-literal-types.html
- **Utility Types**: https://www.typescriptlang.org/docs/handbook/utility-types.html

---

## Coming Up Next: Chapter 19 - Conditional Types

In the next chapter, we will explore:

- Introduction to conditional types (`T extends U ? X : Y`)
- Type inference in conditional types (`infer`)
- Distributive conditional types
- Built-in conditional types: `Exclude`, `Extract`, `NonNullable`
- `ReturnType`, `Parameters`, `InstanceType`, `ThisParameterType`
- Advanced conditional type patterns
- Type-level programming techniques

Conditional types enable type-level logic and are essential for creating sophisticated type utilities that can inspect and transform other types based on their structure.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../5. generics/17. generic_patterns_and_best_practices.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='19. conditional_types.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
